## Packages and libraries installations

Install required packages

In [1]:
!pip install lxml
print('lxml installed')

     |████████████████████████████████| 5.8MB 3.4MB/s eta 0:00:01��█████▎                        | 1.3MB 3.4MB/s eta 0:00:02eta 0:00:01
lxml installed


In [2]:
!conda install -c conda-forge geopy --yes
print('geopy installed')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
print('folium installed')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_1        11.1 MB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-

In [4]:
!pip install geocoder
print('geocoder installed')

     |████████████████████████████████| 102kB 5.7MB/s ta 0:00:011
     |████████████████████████████████| 92kB 6.2MB/s eta 0:00:011
     |████████████████████████████████| 829kB 8.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
geocoder installed


Import required libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# import folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 2. Data Collection

## 2.1. Cities geographical data

### 2.1.1. New York City geographical data

In this section, I need to collect the data for New York City. The data must contain the list of boroughs, neighborhoods, latitude and longitude.

In [6]:
!wget -q -O 'newyork_data.json' https://geo.nyu.edu/download/file/nyu-2451-34572-geojson.json
print('Data downloaded!') 

Data downloaded!


In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

The data is stored in a json file. I need to transform it into a dataframe.

In [8]:
# Store the features data neighborhoods_data
neighborhoods_data = newyork_data['features']

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_neighborhoods_df = pd.DataFrame(columns=column_names)
ny_neighborhoods_df

,Borough,Neighborhood,Latitude,Longitude


Populate the ny_neighborhoods_df dataframe with the features data from the json file.

In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    ny_neighborhoods_df = ny_neighborhoods_df.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
# Examine the resulting dataframe
ny_neighborhoods_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [12]:
ny_neighborhoods_df.shape

(306, 4)

### 2.1.2. New York City Census Data

The next step is to collect the demographic data for each neighborhood. The dataset is obtained from the OpenIPCR website and downloaded as a zip file stored in the local drive. Then the xlsx file is brought into this project.

https://www.openicpsr.org/openicpsr/project/111107/version/V1/view

In [54]:
ny_census_df = pd.read_excel('2010_NY_Census_Data.xlsx', index_col=0)

In [55]:
ny_census_df.reset_index(inplace = True)
ny_census_df.head()

,tract_fips10,totpop10,popden10,phispanic10,pnhwhite10,pnhblack10,pfborn10,ped1_10,ped2_10,ped3_10,pin1b_10,pin2b_10,pin3b_10,pin4b_10,pin5b_10,pincgt75k10,pnvmar10,p18yr_10,p18_2910,p30_3910,p40_4910,p50_6910,pge7010,punemp10,pprof10,ppov10,ppubas10,pfhfam10,pownoc10,disadvantage10,disadvantage2_10,affluence10,ethnicimmigrant10
0,36001000100,2235.0,2445.2950,0.164206,0.184787,0.582998,0.079195,0.288790,0.596140,0.115070,0.139489,0.355599,0.172888,0.222004,0.110020,0.188605,0.411728,0.316778,0.136465,0.133781,0.162863,0.170917,0.079195,0.096671,0.184652,0.279149,0.308594,0.320236,0.466146,0.317529,0.251162,0.162776,0.121700
1,36001000200,5412.0,6790.4640,0.095344,0.105506,0.713784,0.077421,0.240483,0.595534,0.163982,0.400881,0.256388,0.092511,0.151542,0.098678,0.137445,0.608684,0.371951,0.183851,0.112528,0.107908,0.185883,0.037879,0.071155,0.189892,0.413947,0.355209,0.385903,0.223497,0.388000,0.306554,0.163773,0.086382
2,36001000300,6421.0,2876.7920,0.146706,0.371282,0.438561,0.055287,0.229666,0.509385,0.260949,0.248286,0.160701,0.130236,0.303123,0.157654,0.311500,0.492881,0.298863,0.162903,0.137362,0.096870,0.195452,0.108550,0.072235,0.434919,0.294620,0.263454,0.348820,0.416064,0.283538,0.244782,0.335790,0.100997
3,36001000401,2412.0,692.7053,0.022388,0.876866,0.085821,0.061775,0.105263,0.457208,0.437529,0.009501,0.023753,0.083135,0.325416,0.558195,0.793349,0.281806,0.058872,0.110696,0.075871,0.082919,0.245440,0.426202,0.037777,0.581576,0.063876,0.016521,0.054632,0.658892,0.051725,0.043201,0.604151,0.042081
4,36001000403,4540.0,3748.9680,0.015639,0.705727,0.159692,0.192071,0.112569,0.334045,0.553386,0.032901,0.062812,0.102692,0.412762,0.388833,0.594217,0.435431,0.079075,0.287004,0.153745,0.139648,0.207049,0.133480,0.037133,0.497222,0.216079,0.062072,0.086740,0.510269,0.112343,0.100506,0.548275,0.103855


This dataframe contains census data for the entire state of New York, however, I am only interested in the data from the five boroughs in New York City. Moreover, I need to assign community districts to each neighborhood.

In [56]:
tract_puma_df = pd.read_csv('2010_Census_Tract_to_2010_PUMA.txt', dtype=str)

In [57]:
tract_puma_df.head()

,STATEFP,COUNTYFP,TRACTCE,PUMA5CE
0,01,001,020100,02100
1,01,001,020200,02100
2,01,001,020300,02100
3,01,001,020400,02100
4,01,001,020500,02100


In [58]:
# Remove the codes from all the other States. NY State Code is 36.

tract_puma_ny_df = tract_puma_df[tract_puma_df['STATEFP'] == '36']
tract_puma_ny_df.reset_index(inplace = True)  
tract_puma_ny_df.head()

,index,STATEFP,COUNTYFP,TRACTCE,PUMA5CE
0,43195,36,001,000100,02001
1,43196,36,001,000200,02001
2,43197,36,001,000300,02001
3,43198,36,001,000401,02001
4,43199,36,001,000403,02001


The track_fips codes for Bronx, Manhattan, Queens, Staten Island and Brooklyn begin with 36005, 36061, 36081, 36085 and 36047 respectively. Therefore, only those will be extracted.

In [59]:
ny_census_df = pd.concat([tract_puma_ny_df, ny_census_df], axis=1)
ny_census_df.head()

,index,STATEFP,COUNTYFP,TRACTCE,PUMA5CE,tract_fips10,totpop10,popden10,phispanic10,pnhwhite10,pnhblack10,pfborn10,ped1_10,ped2_10,ped3_10,pin1b_10,pin2b_10,pin3b_10,pin4b_10,pin5b_10,pincgt75k10,pnvmar10,p18yr_10,p18_2910,p30_3910,p40_4910,p50_6910,pge7010,punemp10,pprof10,ppov10,ppubas10,pfhfam10,pownoc10,disadvantage10,disadvantage2_10,affluence10,ethnicimmigrant10
0,43195,36,001,000100,02001,3.600100e+10,2235.0,2445.2950,0.164206,0.184787,0.582998,0.079195,0.288790,0.596140,0.115070,0.139489,0.355599,0.172888,0.222004,0.110020,0.188605,0.411728,0.316778,0.136465,0.133781,0.162863,0.170917,0.079195,0.096671,0.184652,0.279149,0.308594,0.320236,0.466146,0.317529,0.251162,0.162776,0.121700
1,43196,36,001,000200,02001,3.600100e+10,5412.0,6790.4640,0.095344,0.105506,0.713784,0.077421,0.240483,0.595534,0.163982,0.400881,0.256388,0.092511,0.151542,0.098678,0.137445,0.608684,0.371951,0.183851,0.112528,0.107908,0.185883,0.037879,0.071155,0.189892,0.413947,0.355209,0.385903,0.223497,0.388000,0.306554,0.163773,0.086382
2,43197,36,001,000300,02001,3.600100e+10,6421.0,2876.7920,0.146706,0.371282,0.438561,0.055287,0.229666,0.509385,0.260949,0.248286,0.160701,0.130236,0.303123,0.157654,0.311500,0.492881,0.298863,0.162903,0.137362,0.096870,0.195452,0.108550,0.072235,0.434919,0.294620,0.263454,0.348820,0.416064,0.283538,0.244782,0.335790,0.100997
3,43198,36,001,000401,02001,3.600100e+10,2412.0,692.7053,0.022388,0.876866,0.085821,0.061775,0.105263,0.457208,0.437529,0.009501,0.023753,0.083135,0.325416,0.558195,0.793349,0.281806,0.058872,0.110696,0.075871,0.082919,0.245440,0.426202,0.037777,0.581576,0.063876,0.016521,0.054632,0.658892,0.051725,0.043201,0.604151,0.042081
4,43199,36,001,000403,02001,3.600100e+10,4540.0,3748.9680,0.015639,0.705727,0.159692,0.192071,0.112569,0.334045,0.553386,0.032901,0.062812,0.102692,0.412762,0.388833,0.594217,0.435431,0.079075,0.287004,0.153745,0.139648,0.207049,0.133480,0.037133,0.497222,0.216079,0.062072,0.086740,0.510269,0.112343,0.100506,0.548275,0.103855


In [60]:
ny_census_df['tract_fips10'].dtypes

dtype('float64')

In [61]:
ny_census_df['tract_fips10'] = ny_census_df['tract_fips10'].astype(str)
ny_census_df['tract_fips10'].dtypes

dtype('O')

In [62]:
ny_census_df.shape

(4919, 38)

In [66]:
ny_census_df.head()

,index,STATEFP,COUNTYFP,TRACTCE,PUMA5CE,tract_fips10,totpop10,popden10,phispanic10,pnhwhite10,pnhblack10,pfborn10,ped1_10,ped2_10,ped3_10,pin1b_10,pin2b_10,pin3b_10,pin4b_10,pin5b_10,pincgt75k10,pnvmar10,p18yr_10,p18_2910,p30_3910,p40_4910,p50_6910,pge7010,punemp10,pprof10,ppov10,ppubas10,pfhfam10,pownoc10,disadvantage10,disadvantage2_10,affluence10,ethnicimmigrant10
0,43195,36,001,000100,02001,36001000100.0,2235.0,2445.2950,0.164206,0.184787,0.582998,0.079195,0.288790,0.596140,0.115070,0.139489,0.355599,0.172888,0.222004,0.110020,0.188605,0.411728,0.316778,0.136465,0.133781,0.162863,0.170917,0.079195,0.096671,0.184652,0.279149,0.308594,0.320236,0.466146,0.317529,0.251162,0.162776,0.121700
1,43196,36,001,000200,02001,36001000200.0,5412.0,6790.4640,0.095344,0.105506,0.713784,0.077421,0.240483,0.595534,0.163982,0.400881,0.256388,0.092511,0.151542,0.098678,0.137445,0.608684,0.371951,0.183851,0.112528,0.107908,0.185883,0.037879,0.071155,0.189892,0.413947,0.355209,0.385903,0.223497,0.388000,0.306554,0.163773,0.086382
2,43197,36,001,000300,02001,36001000300.0,6421.0,2876.7920,0.146706,0.371282,0.438561,0.055287,0.229666,0.509385,0.260949,0.248286,0.160701,0.130236,0.303123,0.157654,0.311500,0.492881,0.298863,0.162903,0.137362,0.096870,0.195452,0.108550,0.072235,0.434919,0.294620,0.263454,0.348820,0.416064,0.283538,0.244782,0.335790,0.100997
3,43198,36,001,000401,02001,36001000401.0,2412.0,692.7053,0.022388,0.876866,0.085821,0.061775,0.105263,0.457208,0.437529,0.009501,0.023753,0.083135,0.325416,0.558195,0.793349,0.281806,0.058872,0.110696,0.075871,0.082919,0.245440,0.426202,0.037777,0.581576,0.063876,0.016521,0.054632,0.658892,0.051725,0.043201,0.604151,0.042081
4,43199,36,001,000403,02001,36001000403.0,4540.0,3748.9680,0.015639,0.705727,0.159692,0.192071,0.112569,0.334045,0.553386,0.032901,0.062812,0.102692,0.412762,0.388833,0.594217,0.435431,0.079075,0.287004,0.153745,0.139648,0.207049,0.133480,0.037133,0.497222,0.216079,0.062072,0.086740,0.510269,0.112343,0.100506,0.548275,0.103855


In [63]:
nyc_census_df = ny_census_df[ny_census_df['tract_fips10'].str.contains("36005") | \
                             ny_census_df['tract_fips10'].str.contains('36061') | \
                             ny_census_df['tract_fips10'].str.contains('36081') | \
                             ny_census_df['tract_fips10'].str.contains('36085') | \
                             ny_census_df['tract_fips10'].str.contains('36047')]

In [64]:
nyc_census_df.shape

(2152, 38)

In [67]:
nyc_census_df.head(32)

,index,STATEFP,COUNTYFP,TRACTCE,PUMA5CE,tract_fips10,totpop10,popden10,phispanic10,pnhwhite10,pnhblack10,pfborn10,ped1_10,ped2_10,ped3_10,pin1b_10,pin2b_10,pin3b_10,pin4b_10,pin5b_10,pincgt75k10,pnvmar10,p18yr_10,p18_2910,p30_3910,p40_4910,p50_6910,pge7010,punemp10,pprof10,ppov10,ppubas10,pfhfam10,pownoc10,disadvantage10,disadvantage2_10,affluence10,ethnicimmigrant10
88,43283,36,005,026701,03701,36005000100.0,9741.0,15968.850,0.339185,0.066831,0.562981,0.158608,0.689461,0.290105,0.020434,NaN,NaN,NaN,NaN,NaN,NaN,0.843343,0.061082,0.458885,0.202033,0.188482,0.086028,0.003490,0.000000,NaN,NaN,NaN,NaN,NaN,0.281491,0.000000,0.020434,0.248896
89,43284,36,005,026702,03701,36005000200.0,4734.0,26897.730,0.658428,0.005281,0.261301,0.255809,0.203505,0.596209,0.200286,0.077909,0.102564,0.148915,0.303748,0.366864,0.544379,0.495948,0.245247,0.245247,0.091677,0.174694,0.180820,0.062315,0.061268,0.257727,0.179732,0.186493,0.156805,0.633154,0.169120,0.146074,0.334131,0.457119
90,43285,36,005,027300,03701,36005000400.0,5211.0,14803.980,0.664940,0.026290,0.256956,0.233928,0.169139,0.551911,0.278950,0.063481,0.038516,0.204708,0.306705,0.386591,0.505706,0.481349,0.232969,0.167722,0.165611,0.165611,0.209557,0.058530,0.037800,0.342566,0.080723,0.114744,0.200428,0.712247,0.138130,0.108424,0.375741,0.449434
91,43286,36,005,027700,03701,36005001600.0,5127.0,27417.110,0.652038,0.017944,0.324751,0.280281,0.302869,0.569062,0.128069,0.073279,0.267950,0.305700,0.259067,0.094004,0.225018,0.468869,0.245953,0.152721,0.103959,0.142969,0.237176,0.117222,0.019990,0.194320,0.176434,0.272165,0.322724,0.212887,0.223213,0.197828,0.182469,0.466160
92,43287,36,005,027900,03701,36005001900.0,2420.0,3811.024,0.591322,0.125620,0.257438,0.289256,0.291235,0.577640,0.131125,0.224891,0.270742,0.248908,0.174672,0.080786,0.150655,0.589425,0.268182,0.204959,0.215289,0.099174,0.170248,0.042149,0.086796,0.211449,0.465459,0.369127,0.233624,0.079195,0.282489,0.288752,0.164410,0.440289
93,43288,36,005,028100,03701,36005002000.0,8523.0,55705.880,0.655051,0.007040,0.306817,0.201220,0.383012,0.491152,0.125836,0.302557,0.251420,0.161458,0.152462,0.132102,0.173295,0.509872,0.283586,0.164144,0.108999,0.153467,0.224803,0.065001,0.053172,0.207847,0.376075,0.484546,0.378788,0.136258,0.319880,0.323145,0.168993,0.428136
94,43289,36,005,028300,03701,36005002300.0,4779.0,119475.000,0.725466,0.020297,0.254237,0.191044,0.516382,0.440273,0.043345,0.441882,0.278598,0.100553,0.159594,0.019373,0.125461,0.614105,0.274325,0.159029,0.096464,0.174932,0.168655,0.126595,0.056508,0.139553,0.527318,0.635938,0.481550,0.004167,0.391110,0.425328,0.102786,0.458255
95,43290,36,005,028500,03701,36005002500.0,4785.0,97653.060,0.788506,0.026959,0.158830,0.340230,0.495351,0.468169,0.036481,0.395183,0.297056,0.117752,0.172168,0.017841,0.064228,0.533466,0.303448,0.200000,0.168025,0.094671,0.148798,0.085058,0.083021,0.096257,0.514352,0.536692,0.346120,0.022453,0.327803,0.370046,0.065655,0.564368
96,43291,36,005,028700,03701,36005002701.0,3100.0,96874.990,0.601935,0.000000,0.398064,0.185161,0.429441,0.435528,0.135030,0.289024,0.262195,0.190244,0.181707,0.076829,0.186585,0.598742,0.293226,0.146452,0.150323,0.156452,0.193548,0.060000,0.101789,0.172545,0.442258,0.608159,0.332927,0.000000,0.376640,0.371283,0.164720,0.393548
97,43292,36,005,028900,03701,36005002702.0,4703.0,77098.360,0.755475,0.003827,0.228152,0.329364,0.499160,0.431092,0.069748,0.336174,0.330492,0.282197,0.030303,0.020833,0.028409,0.604862,0.332766,0.224325,0.112907,0.179673,0.097810,0.052520,0.074865,0.168084,0.466386,0.512286,0.414773,0.012658,0.339292,0.367077,0.088747,0.542420


Now, based on the information from the census data set, neighborhoods are not explicitly reported on but the data is reported on the level of Public Use Microdata Area (PUMA). Each PUMA may encompass two or more neighborhoods. Therefore I need to extract the census data for the neighborhoods and fit it into the geographical dataframe.

Define Foursquare credentials

In [ ]:
CLIENT_ID = 'A0J0GDHGCDD1YJNKW5H15WRHYLDYQM2KXG25Z5E2KLSSYLTE' # your Foursquare ID
CLIENT_SECRET = '33KHVJ1UNG3XLU0CHA31L2KPP0ITUTYJD1HEN4MY121E5GXM' # your Foursquare Secret
VERSION = '20200422' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)